1. Fix logger for DDP, something to do with computing torchmetrics on a single node 
2. Redirect [all] warnings to logfile
3. Use importlib and getattr to use classnames / function names directly in config.yaml 
4. Add options to change mode [fit, validate, test], select config.yaml, select profiling, delete logs dir before starting, to run.py

5. Add spatial tiling -> [x] image level 
6. Add polygonization
7. Add NCut Loss

In [1]:
%load_ext autoreload
%load_ext memory_profiler 
%load_ext dotenv
%autoreload 2
%dotenv

In [2]:
import pandas as pd
from geovision.data import DatasetConfig
from geovision.data.inria import Inria

index_df = Inria.load('index', 'imagefolder', 'inria')
spatial_df = Inria.load('spatial', 'imagefolder', 'inria')
#index_df.merge(spatial_df, how = 'left', left_index=True, right_index=True)

In [5]:
tile_size = 256
tile_stride = 256
df = DatasetConfig.sliding_window_spatial_sampler(index_df, spatial_df, tile_size, tile_stride)
df = df.merge(spatial_df, 'left', left_index=True, right_index=True)
df["image_height"] = tile_size 
df["image_width"] = tile_size 
df["x_off"] = df["x_off"] + df["tile_tl_0"] * df["x_res"]
df["y_off"] = df["y_off"] + df["tile_tl_1"] * df["y_res"]
df

,image_path,mask_path,location,tile_tl_0,tile_tl_1,tile_br_0,tile_br_1,image_height,image_width,x_off,y_off,x_res,y_res,crs
0,images/austin1.tif,masks/austin1.tif,austin,0,0,256,256,256,256,616500.0,3345000.0,0.3,-0.3,26914
0,images/austin1.tif,masks/austin1.tif,austin,0,256,256,512,256,256,616500.0,3344923.2,0.3,-0.3,26914
0,images/austin1.tif,masks/austin1.tif,austin,0,512,256,768,256,256,616500.0,3344846.4,0.3,-0.3,26914
0,images/austin1.tif,masks/austin1.tif,austin,0,768,256,1024,256,256,616500.0,3344769.6,0.3,-0.3,26914
0,images/austin1.tif,masks/austin1.tif,austin,0,1024,256,1280,256,256,616500.0,3344692.8,0.3,-0.3,26914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,images/vienna9.tif,masks/vienna9.tif,vienna,4864,3840,5120,4096,256,256,2235.2,341759.0,0.3,-0.3,31256
179,images/vienna9.tif,masks/vienna9.tif,vienna,4864,4096,5120,4352,256,256,2235.2,341682.2,0.3,-0.3,31256
179,images/vienna9.tif,masks/vienna9.tif,vienna,4864,4352,5120,4608,256,256,2235.2,341605.4,0.3,-0.3,31256
179,images/vienna9.tif,masks/vienna9.tif,vienna,4864,4608,5120,4864,256,256,2235.2,341528.6,0.3,-0.3,31256


In [ ]:
import logging
from lightning import Trainer
from geovision.experiment.config import ExperimentConfig
from geovision.models.interfaces import ClassificationModule
from geovision.data import ImageDatasetDataModule

from geovision.io.local import FileSystemIO as fs
from geovision.experiment.loggers import (
    get_csv_logger, 
    get_ckpt_logger,
    get_classification_logger
)

In [ ]:
config = ExperimentConfig.from_yaml("config.yaml")
datamodule = ImageDatasetDataModule(config.dataset_constructor, config.dataset_config, config.dataloader_config)

logging.basicConfig(
    filename = f"{fs.get_new_dir(config.experiments_dir, "logs")}/logfile.log",
    filemode = "a",
    format = "%(asctime)s : %(name)s : %(levelname)s : %(message)s",
    level = logging.INFO
)

loggers: list = list()
loggers.append(csv_logger := get_csv_logger(config))

callbacks: list = list()
#callbacks.append(ckpt_logger := get_ckpt_logger(config))
callbacks.append(metrics_logger := get_classification_logger(config))

In [ ]:
config = ExperimentConfig.from_yaml("config.yaml")
model = ClassificationModule(
    model_config=config.model_config,
    criterion_constructor=config.criterion_constructor, 
    criterion_params=config.criterion_params, 
    optimizer_constructor=config.optimizer_constructor,
    optimizer_params=config.optimizer_params,
    lr_scheduler_constructor=config.scheduler_constructor,
    lr_scheduler_params=config.scheduler_params,
    warmup_scheduler_constructor=config.warmup_scheduler_constructor,
    warmup_scheduler_params=config.warmup_scheduler_params,
    warmup_steps=config.warmup_steps,
    scheduler_config_params=config.scheduler_config_params
)
trainer = Trainer(logger = loggers, callbacks = callbacks, **config.trainer_params)
trainer.fit(
    model = model, 
    datamodule = datamodule, 
    ckpt_path = config.ckpt_path 
)